# Aviation Risk Analysis

### Overview

As the company expands into the aviation industry, understanding the risks associated with different aircraft models is crucial. This project analyzes historical aviation incident data to identify low-risk aircraft for commercial and private use.

### Objective
- Determine which aircraft models have the lowest accident rates.
- Identify key risk factors such weather, flight phase, aircraft manufacturer.
- Provide three concrete business recommendations to guide aircraft purchase decisions.

### Business Problem
Your company is expanding in to new industries to diversify its portfolio. Specifically, they are interested in purchasing and operating airplanes for commercial and private enterprises, but do not know anything about the potential risks of aircraft. You are charged with determining which aircraft are the lowest risk for the company to start this new business endeavor. You must then translate your findings into actionable insights that the head of the new aviation division can use to help decide which aircraft to purchase.

In [29]:
# import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [30]:
# load and preview the dataset 
aviation_data_df = pd.read_csv("data/AviationData.csv", encoding='latin-1', index_col=0, low_memory=False)
aviation_data_df.head()

,Investigation.Type,Accident.Number,Event.Date,Location,Country,Latitude,Longitude,Airport.Code,Airport.Name,Injury.Severity,...,Purpose.of.flight,Air.carrier,Total.Fatal.Injuries,Total.Serious.Injuries,Total.Minor.Injuries,Total.Uninjured,Weather.Condition,Broad.phase.of.flight,Report.Status,Publication.Date
Event.Id,,,,,,,,,,,,,,,,,,,,,
20001218X45444,Accident,SEA87LA080,1948-10-24,"MOOSE CREEK, ID",United States,NaN,NaN,NaN,NaN,Fatal(2),...,Personal,NaN,2.0,0.0,0.0,0.0,UNK,Cruise,Probable Cause,NaN
20001218X45447,Accident,LAX94LA336,1962-07-19,"BRIDGEPORT, CA",United States,NaN,NaN,NaN,NaN,Fatal(4),...,Personal,NaN,4.0,0.0,0.0,0.0,UNK,Unknown,Probable Cause,19-09-1996
20061025X01555,Accident,NYC07LA005,1974-08-30,"Saltville, VA",United States,36.922223,-81.878056,NaN,NaN,Fatal(3),...,Personal,NaN,3.0,NaN,NaN,NaN,IMC,Cruise,Probable Cause,26-02-2007
20001218X45448,Accident,LAX96LA321,1977-06-19,"EUREKA, CA",United States,NaN,NaN,NaN,NaN,Fatal(2),...,Personal,NaN,2.0,0.0,0.0,0.0,IMC,Cruise,Probable Cause,12-09-2000
20041105X01764,Accident,CHI79FA064,1979-08-02,"Canton, OH",United States,NaN,NaN,NaN,NaN,Fatal(1),...,Personal,NaN,1.0,2.0,NaN,0.0,VMC,Approach,Probable Cause,16-04-1980


In [31]:
# Display basic information about the dataset
aviation_data_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 88889 entries, 20001218X45444 to 20221230106513
Data columns (total 30 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Investigation.Type      88889 non-null  object 
 1   Accident.Number         88889 non-null  object 
 2   Event.Date              88889 non-null  object 
 3   Location                88837 non-null  object 
 4   Country                 88663 non-null  object 
 5   Latitude                34382 non-null  object 
 6   Longitude               34373 non-null  object 
 7   Airport.Code            50132 non-null  object 
 8   Airport.Name            52704 non-null  object 
 9   Injury.Severity         87889 non-null  object 
 10  Aircraft.damage         85695 non-null  object 
 11  Aircraft.Category       32287 non-null  object 
 12  Registration.Number     87507 non-null  object 
 13  Make                    88826 non-null  object 
 14  Model                

#### Determine the shape of the data set

In [33]:
# Display basic statistics about the dataset
aviation_data_df.shape

(88889, 30)

Model
152                    2367
172                    1756
172N                   1164
PA-28-140               932
150                     829
                       ... 
QUESTAIR VENTURE LX       1
310M                      1
BCS12-D                   1
112-TC                    1
MH-1521M                  1
Name: count, Length: 12318, dtype: int64

The dataset consists of 30 columns and 88889 rows

### Data Cleaning 
Here, the data is being cleaned, handling missing values and dropping irrelevant rows.

In [35]:
aviation_data_df.isnull().sum()

Investigation.Type            0
Accident.Number               0
Event.Date                    0
Location                     52
Country                     226
Latitude                  54507
Longitude                 54516
Airport.Code              38757
Airport.Name              36185
Injury.Severity            1000
Aircraft.damage            3194
Aircraft.Category         56602
Registration.Number        1382
Make                         63
Model                        92
Amateur.Built               102
Number.of.Engines          6084
Engine.Type                7096
FAR.Description           56866
Schedule                  76307
Purpose.of.flight          6192
Air.carrier               72241
Total.Fatal.Injuries      11401
Total.Serious.Injuries    12510
Total.Minor.Injuries      11933
Total.Uninjured            5912
Weather.Condition          4492
Broad.phase.of.flight     27165
Report.Status              6384
Publication.Date          13771
dtype: int64

In [46]:
# clean the dataset
# convert Event.Date to datetime
aviation_data_df["Event.Date"] = pd.to_datetime(aviation_data_df["Event.Date"])

# convert Injury.Severity to numeric
numerical_columns = ['Total.Fatal.Injuries', 'Total.Serious.Injuries', 'Total.Minor.Injuries', 'Total.Uninjured', 'Number.of.Engines']
aviation_data_df[numerical_columns] = aviation_data_df[numerical_columns].apply(pd.to_numeric, errors='coerce')

aviation_data_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 88889 entries, 20001218X45444 to 20221230106513
Data columns (total 30 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   Investigation.Type      88889 non-null  object        
 1   Accident.Number         88889 non-null  object        
 2   Event.Date              88889 non-null  datetime64[ns]
 3   Location                88837 non-null  object        
 4   Country                 88663 non-null  object        
 5   Latitude                34382 non-null  object        
 6   Longitude               34373 non-null  object        
 7   Airport.Code            50132 non-null  object        
 8   Airport.Name            52704 non-null  object        
 9   Injury.Severity         0 non-null      float64       
 10  Aircraft.damage         85695 non-null  object        
 11  Aircraft.Category       32287 non-null  object        
 12  Registration.Number     87507

In [50]:
# Clean Model and Make columns
aviation_data_df['Model'] = aviation_data_df['Model'].str.strip().str.upper()
aviation_data_df['Make'] = aviation_data_df['Make'].str.strip().str.upper()

# This will ensure that there are no duplicated values in the Make column
aviation_data_df['Make'].value_counts()

Make
CESSNA           27149
PIPER            14870
BEECH             5372
BOEING            2745
BELL              2722
                 ...  
IZATT                1
MINCE                1
DANA A. MOORE        1
SLATER               1
SCOVIL               1
Name: count, Length: 7587, dtype: int64

#### Drop missing values 

In [74]:
cleaned_data_df = aviation_data_df.dropna(subset=['Model', 'Make', 'Location'])
cleaned_data_df.shape


(88725, 30)

After dropping missing values for make and model, we now have 88777 rows

In [75]:
# cleaned_data_df["Total.Fatal.Injuries"].isnull().sum()
cleaned_data_df["Purpose.of.flight"].fillna("Unknown", inplace=True)
cleaned_data_df["Purpose.of.flight"].unique()

/tmp/ipykernel_707919/1668497785.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  cleaned_data_df["Purpose.of.flight"].fillna("Unknown", inplace=True)
/tmp/ipykernel_707919/1668497785.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned_data_df["Purpose.of.flight"].fillna("Unknown", inplace=True)


array(['Personal', 'Unknown', 'Business', 'Instructional', 'Ferry',
       'Executive/corporate', 'Aerial Observation', 'Aerial Application',
       'Public Aircraft', 'Skydiving', 'Other Work Use', 'Positioning',
       'Flight Test', 'Air Race/show', 'Air Drop',
       'Public Aircraft - Federal', 'Glider Tow',
       'Public Aircraft - Local', 'External Load',
       'Public Aircraft - State', 'Banner Tow', 'Firefighting',
       'Air Race show', 'PUBS', 'ASHO', 'PUBL'], dtype=object)

In [76]:
# Clean Location column
cleaned_data_df['Location'] = cleaned_data_df['Location'].str.strip().str.upper()
cleaned_data_df['Location'].value_counts()

/tmp/ipykernel_707919/564617180.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned_data_df['Location'] = cleaned_data_df['Location'].str.strip().str.upper()


Location
ANCHORAGE, AK      548
MIAMI, FL          275
HOUSTON, TX        271
ALBUQUERQUE, NM    265
CHICAGO, IL        255
                  ... 
DELHI,               1
IGARASSU,            1
KITCHENER,           1
LIBERIA,             1
TANNER, AL           1
Name: count, Length: 21945, dtype: int64

#### Data Visualization